# 基于 VERL 的Reasoning Reward Model训练指南

## 📖 概述

本文档提供了使用 VERL 框架训练Reasoning Reward Model的完整指南。通过本教程，您将学会如何配置环境、准备数据、设计奖励函数并执行训练流程。

## 🏗️ 系统架构

### 核心组件

VERL 推理奖励模型训练系统由以下三个核心组件构成：

#### 1. **训练数据集** - 继承自 `BaseTrainDataset`
   - 支持多种数据格式和评估标准
   - 提供灵活的对话模板系统
   - 集成自定义奖励函数

#### 2. **提示模板** - 基于 `BasePromptTemplate`
   - 定义结构化输出格式
   - 支持可扩展的评分标准
   - 适配多种任务类型

#### 3. **奖励函数** - 可自定义的奖励计算模块
   - 支持 pointwise、pairwise 等多种奖励计算方式
   - 提供灵活的评估指标配置
   - 实时统计准确率和奖励分数

## 🔧 环境配置

### 运行环境要求

创建 `runtime_env.yaml` 配置文件：

```yaml
# runtime_env.yaml
excludes: ["/.git/"]
env_vars:
  TORCH_NCCL_AVOID_RECORD_STREAMS: "1"
  PYTORCH_CUDA_ALLOC_CONF: "expandable_segments: False"
  WANDB_API_KEY: "your_wandb_api_key"
  WANDB_BASE_URL: "your_wandb_base_url"
  HYDRA_FULL_ERROR: "1"
```

### 依赖安装

确保安装以下必要依赖：
- `verl==0.4.0` (核心框架)


## 🚀 快速开始

### 第一步：准备训练数据

训练数据应符合 `DataSample` 格式规范。详细的数据加载和预处理步骤请参考数据加载章节。

### 第二步：启动 Ray 分布式集群

#### 主节点 (Master) 启动：
```bash
ray start --head --node-ip-address $MASTER_ADDR --num-gpus 8
```

#### 工作节点 (Workers) 启动：
```bash
ray start --address=$MASTER_ADDR:6379 --num-gpus 8
```

### 第三步：执行训练流程

```bash
# 进入训练目录
cd rm_gallery/gallery/train/<your_method>

# 启动训练脚本
bash run_train.sh
```

### 数据格式说明

- **输入数据格式**：所有输入数据必须符合 `DataSample` 格式

## 🧩 核心组件详解

### 自定义训练数据集

以下是自定义训练数据集的完整实现示例：

```python
class CustomTrainDataset(BaseTrainDataset):
    def __init__(self, *args, **kwargs):
        # 初始化奖励模块
        self.reward_module = YourRewardModule(
            name="custom_reward",
            template=YourTemplate,
            examples=self._get_examples(),
            llm=None,
        )
        super().__init__(*args, **kwargs)
    
    def _build_messages(self, example):
        # 构建格式化消息
        result = self.reward_module.format(sample=example)
        return [{"role": "user", "content": result}]
```

> **重要提示：推理模型配置**
> 
> 训练推理奖励模型时，需要注意以下配置：
> - 当基础模型为推理模型（如 Qwen3）时：
>   - `apply_chat_template` 的 `enable_thinking=True`
>   - `format` 的 `enable_thinking=False`
> - 对于非推理模型：
>   - `apply_chat_template` 的 `enable_thinking=False`
>   - `format` 的 `enable_thinking=True`
>
> ```python
> # 推理模型配置示例
> self.tokenizer.apply_chat_template(
>     messages, add_generation_prompt=True, tokenize=False, enable_thinking=True
> )
> 
> result = self.helpfulness_reward.format(sample=example, enable_thinking=False)
> ```

### 奖励函数设计

奖励函数是评估模型性能的关键组件：

```python
def calculate_reward(predicted_score, true_score):
    """
    自定义奖励函数
    
    Args:
        predicted_score (float): 模型预测分数
        true_score (float): 真实标签分数
        
    Returns:
        float: 计算得到的奖励值
    """
    if true_score is None:
        return 0.0
    
    # 基于绝对误差的奖励计算
    abs_error = abs(predicted_score - true_score)
    max_error = 4  # 根据评分范围调整 (例如：0-4分制)
    
    # 线性衰减奖励函数
    reward = 1.0 - (abs_error / max_error)
    return max(0.0, reward)  # 确保奖励值非负
```

### 提示模板系统

模板系统定义了模型输入输出的结构化格式：

```python
class YourTemplate(BasePromptTemplate):
    score: int = Field(description="评分结果 (0-4分)")
    
    @classmethod
    def format(cls, desc, examples, query, answer, **kwargs):
        """
        格式化提示模板
        
        Args:
            desc (str): 任务描述
            examples (str): 示例数据
            query (str): 用户查询
            answer (str): 模型回答
        
        Returns:
            str: 格式化后的提示文本
        """
        return f"""# 任务描述
{desc}

# 参考示例
{examples}

# 用户查询
{query}

# 模型回答
{answer}

# 输出要求
{cls.schema(**kwargs)}
        """
```


## 💡 实现示例：Pointwise 训练

### 项目结构

```
rm_gallery/gallery/train/pointwise/
├── run_pointwise.sh    # 训练启动脚本
├── dataset.py          # 数据集实现
├── reward_fn.py        # 奖励函数实现
├── template.py         # 提示模板
└── runtime_env.yaml    # 训练配置文件
```

### 实验配置

#### 数据集选择
- **数据源**：`nvidia/helpsteer2` 数据集
- **评估维度**：`helpfulness` 标注信息
- **数据规模**：训练集约 8K 样本，验证集约 8K 样本
- **评分范围**：0-4 分制 (0=最差，4=最佳)

#### 模型配置
- **基础模型**：`Qwen3-8B`

#### 训练参数
详细训练参数配置请查看：`./gallery/train/pointwise/run_pointwise.sh`

### Pointwise 训练数据集

```python
class PointwiseTrainDataset(BaseTrainDataset):
    """
    Pointwise 训练数据集实现
    
    该数据集专门用于Pointwise评分任务，对每个样本独立进行质量评估
    """
    def __init__(self, *args, **kwargs):
        self.reward_module = PointwiseReward(
            name="pointwise_reward",
            template=PointwiseTemplate,
            examples=self._get_examples(),
            llm=None,
        )
        super().__init__(*args, **kwargs)
```

### Pointwise 奖励函数

```python
import math

def pointwise_reward(predicted_score, true_score):
    """
    点式评分奖励函数
    
    采用指数衰减机制，对预测误差进行惩罚
    
    Args:
        predicted_score (float): 模型预测的分数
        true_score (float): 真实的标准分数
        
    Returns:
        float: 计算得到的奖励值 [0, 1]
    """
    if true_score is None:
        return 0.0
    
    # 计算绝对误差
    abs_error = abs(predicted_score - true_score)
    max_error = 4  # 评分范围 0-4
    
    # 指数衰减奖励计算
    k = 2.0  # 衰减系数，控制惩罚强度
    error_ratio = abs_error / max_error
    reward = math.exp(-k * error_ratio)
    
    return float(reward)
```

### 训练结果分析

#### 训练曲线

我们通过训练和验证曲线来评估模型的学习效果：

**训练奖励曲线**：
![训练奖励曲线](../../images/data/pointwise_train.jpg)

**验证奖励曲线**：
![验证奖励曲线](../../images/data/pointwise_val.jpg)

## 🔗 相关资源

### 框架文档
- **VERL 框架**: https://github.com/volcengine/verl - 核心训练框架
- **Ray 分布式**: https://docs.ray.io/ - 分布式计算平台
- **VLLM 推理**: https://docs.vllm.ai/ - 高性能推理引擎

### 数据集资源
- **HelpSteer2**: https://huggingface.co/datasets/nvidia/helpsteer2 - 人类偏好数据集
- **UltraFeedback**: https://huggingface.co/datasets/openbmb/UltraFeedback - 多维度反馈数据
